Setup cell

In [ ]:
!pip3 install pgmpy
import pgmpy
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

!pip3 install owlready2
import owlready2 as owl

HOMO_DOM = 0
HETERO = 1
HOMO_REC = 2

Importing ontology. Unfortunately, due to syntactic interoperability concerns *after* importation, using this ontology to create the Bayesian network proved to be impossible. Though the ontology individuals were able to be listed, accessing them programatically and crawling over the resultant graph proved impossible.

In [12]:
onto_filepath = "20220224_inheritable_disease_ontology_v1.0.owl"
onto = owl.get_ontology("file://" + onto_filepath)
onto.load()
ont = onto.get_namespace("file://" + onto_filepath)
# print(list(onto.individuals()))

The `create_network` function showing pedigree creation for a proband generation with a variable number of children. A pseudocode version of this function that would work recursively is found at the end of the notebook.

In [ ]:
def create_network(recessive_baseline_freq, n_children):
  parents = ["I-1", "I-2"]
  children = []
  for i in range(1, n_children + 1):
    children.append("II-"+str(i))
  
  bayesNet = BayesianNetwork()

  bayesNet.add_nodes_from(parents)
  bayesNet.add_nodes_from(children)

  for parent in parents:
    for child in children:
      bayesNet.add_edge(parent, child)

  for child in children:
    bayesNet.add_cpds(
        TabularCPD(
          child, 3, 
          [[1, 0.5, 0, 0.5, 0.25, 0, 0, 0 ,0],
                  [0, 0.5, 1, 0.5, 0.5, 0.5, 1, 0.5, 0],
                  [0, 0, 0, 0, 0.25, 0.5, 0, 0.5, 1]],
                 parents, [3,3])
    )

  naive_parental_genotype = [[(1 - recessive_baseline_freq) ** 2], 
                             [2 * (1 - recessive_baseline_freq) *(recessive_baseline_freq)], 
                             [(recessive_baseline_freq ** 2)]]
  for parent in parents:
    bayesNet.add_cpds(
        TabularCPD(parent, 3, values=naive_parental_genotype)
    )

  return bayesNet

Creating the model, then querying it for results.

In [6]:
BASELINE_REC_ALLELE_FREQ = 0.002
model = create_network(BASELINE_REC_ALLELE_FREQ, n_children=3)
model.check_model()
solver = VariableElimination(model)

In [ ]:
figure8 = solver.query(variables=['I-1'], 
                        evidence={'I-2': HETERO, 
                                  'II-1': HETERO, 
                                  'II-3': HOMO_REC })
print(figure8)

In [8]:
figure9 = solver.query(variables=['II-2'], 
                        evidence={'I-2': HETERO, 
                                  'II-1': HETERO, 
                                  'II-3': HOMO_REC })
print(figure9)

Eliminating: I-1: 100%|██████████| 1/1 [00:00<00:00, 479.57it/s]

+---------+-------------+
| II-2    |   phi(II-2) |
+=========+=============+
| II-2(0) |      0.2495 |
+---------+-------------+
| II-2(1) |      0.5000 |
+---------+-------------+
| II-2(2) |      0.2505 |
+---------+-------------+


Pseudocode for a recursive implementation of the `create_network`, that would travel a pedigree imported from the OWL ontology. Note that this code cannot be run due to issues programatticaly accessing the individuals imported from the OWL ontology. 

In [ ]:
#find_spouse is a helper function, since a hasSpouse relation is not existing in our ontology but can be implied through person -> hasChild -> hasParent relationships. This same inference could be done with an ontology reasoner.
def find_spouse(person):
    children = person.hasChildren()
    spouse = NULL
    if children:
        parents = children[0].hasParent()
    for parent in parents:
        if parent != person:
            spouse = parent
    return spouse

global bayesNet
bayesNet = BayesianNetwork()
def create_network(root, parental_generation, parent_index, 
                    starting_child_index, n_children, recessive_baseline_freq):
  # known edge case - consanguinous marriage
  
  parents = [root, find_spouse(root)]
  generation_index = "I" * parental_generation
  for i in range(2):
    parents.append(generation_index + "-" + str(parent_index + i))
  
  children = []
  for i in range(1, n_children + 1):
    children.append(generation_index + "I-" +str(starting_child_index + i))

  bayesNet.add_nodes_from(parents)
  bayesNet.add_nodes_from(children)

  grandkidCounter = 0
  for child in children:
    for parent in parents:
      bayesNet.add_edge(parent, child)
    if child.hasChild():
        create_network(bayesNet, child, 
                          parental_generation = parental_generation + 1,
                          parent_index = child.slice("-")[1] #get index of child
                          starting_child_index = grandkidCounter + len(child.hasChildren()),
                          n_children= len(child.hasChildren()),
                          recessive_baseline_freq = recessive_baseline_freq
        )

  for child in children:
    bayesNet.add_cpds(
        TabularCPD(
          child, 3, 
          [[1, 0.5, 0, 0.5, 0.25, 0, 0, 0 ,0],
                  [0, 0.5, 1, 0.5, 0.5, 0.5, 1, 0.5, 0],
                  [0, 0, 0, 0, 0.25, 0.5, 0, 0.5, 1]],
                 parents, [3,3])
    )

  naive_parental_genotype = [[(1 - recessive_baseline_freq) ** 2], 
                             [2 * (1 - recessive_baseline_freq) *(recessive_baseline_freq)], 
                             [(recessive_baseline_freq ** 2)]]
  for parent in parents:
    bayesNet.add_cpds(
        TabularCPD(parent, 3, values=naive_parental_genotype)
    )

  return starting_child_index + n_children